In [31]:
import requests
import base64
import json
import nacl.secret
import nacl.utils
import nacl.pwhash
from nacl.signing import SigningKey
from nacl.encoding import Base64Encoder
import nacl.encoding
import nacl.hash
from nacl.public import PrivateKey, Box


# q5p1

In [30]:
myheaders = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

url2 =  "https://hash-browns.cs.uwaterloo.ca/api/pke/get-key"

myobj2 = {"api_token": "cb0fc431d5b7e268e4a81538dd653b83ad0702876a367b628ba6ec7ceb6800d0",
        "user": "Gooser"
         }

r = requests.post(url2, json = myobj2, headers=myheaders)
r.status_code

data = json.loads(r.text)
# print (data)
pubkey = data['pubkey']
# decodedmsg = base64.b64decode(msg)
# plaintext = box.decrypt(decodedmsg)

print (r.status_code, r.text)
print(pubkey)

#---------------------------------------------

decodedmsg = base64.b64decode(pubkey)

hasher = nacl.hash.blake2b
hashed = hasher(decodedmsg, encoder=nacl.encoding.HexEncoder)

print ('hash of goosers public string', hashed)


200 {"user":"Gooser","pubkey":"l19VCKk31NjSkHPTrFp3FPEA4GEK6ooLYl6Wwobc4Gg="}
l19VCKk31NjSkHPTrFp3FPEA4GEK6ooLYl6Wwobc4Gg=
hash of goosers public string b'ca48e2d7b7841b59a75730ec97e77f808d7a5ea73b6d39f6409f18841170290f'


# q5p2

In [44]:
#get goosers pub key

#i am bob
# Generate Bob's private key, which must be kept secret
skbob = PrivateKey.generate()

# Bob's public key can be given to anyone wishing to send
#   Bob an encrypted message
pkbob = skbob.public_key



url =  "https://hash-browns.cs.uwaterloo.ca/api/pke/set-key"

myheaders = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}


# byte_key = bytes.fromhex(givenkey)

# box = nacl.secret.SecretBox(byte_key)
# encoded = base64.b64encode(box.encrypt(mymsg))

print (type(pkbob))
pkbob = pkbob.encode(Base64Encoder)
pkbob = pkbob.decode()
print ('pubkey', pkbob)


myobj = {"api_token": "cb0fc431d5b7e268e4a81538dd653b83ad0702876a367b628ba6ec7ceb6800d0",
         "pubkey": pkbob
         }


r = requests.post(url, json = myobj, headers=myheaders)
r.status_code
print (r.status_code)


<class 'nacl.public.PublicKey'>
pubkey swcoIFU2xIYc66ChqcTkOvv+im3XPpqb3j+aulWscCw=
200
b'ca48e2d7b7841b59a75730ec97e77f808d7a5ea73b6d39f6409f18841170290f'


In [67]:
#sending my encrypted message

#im now bob
print (hashed)
print (type(skbob), type(decodedmsg))
bob_box = Box(skbob, nacl.public.PublicKey(decodedmsg))

message = b'wassup gooser it works'
encrypted = bob_box.encrypt(message)
encrypted = base64.b64encode(encrypted)
encrypted = encrypted.decode()
print ('encrypted:', encrypted)

url3 =  "https://hash-browns.cs.uwaterloo.ca/api/pke/send"

myobj3 = {"api_token": "cb0fc431d5b7e268e4a81538dd653b83ad0702876a367b628ba6ec7ceb6800d0",
        "recipient": "Gooser",
        "msg": encrypted
         }

r = requests.post(url3, json = myobj3, headers=myheaders)
r.status_code
print (r.status_code)

b'ca48e2d7b7841b59a75730ec97e77f808d7a5ea73b6d39f6409f18841170290f'
<class 'nacl.public.PrivateKey'> <class 'bytes'>
encrypted: Z+1/gH0ZGRPXQWYBTODSjLx+CAam5H9VgIkPr/n2RG39tG1mhrePacU1X+0FViyy9vl2PIPVWqAlC75A5MA=
200


# q5p3


In [71]:
url4 =  "https://hash-browns.cs.uwaterloo.ca/api/pke/inbox"

myobj4 = {"api_token": "cb0fc431d5b7e268e4a81538dd653b83ad0702876a367b628ba6ec7ceb6800d0",
         }

r = requests.post(url4, json = myobj4, headers=myheaders)
r.status_code

data = json.loads(r.text)
msg = data[0]['msg']
#print ('msg:', msg)
decodedmsg = base64.b64decode(msg)

print (r.status_code, r.text)
print(decodedmsg)

# using my original bob box
# Decrypt our message, an exception will be raised if the encryption was
#   tampered with or there was otherwise an error.
plaintext = bob_box.decrypt(decodedmsg)
print(plaintext.decode('utf-8'))


200 [{"msg_id":1073741824,"sender":"Gooser","msg":"NObwjtSsv8vGNQVulslt3+hEmjJ5vGROQe9avwiG5grWU0oVZ4nAckVz34OBy8cKAsYDqIk0Hw=="}]
b'4\xe6\xf0\x8e\xd4\xac\xbf\xcb\xc65\x05n\x96\xc9m\xdf\xe8D\x9a2y\xbcdNA\xefZ\xbf\x08\x86\xe6\n\xd6SJ\x15g\x89\xc0rEs\xdf\x83\x81\xcb\xc7\n\x02\xc6\x03\xa8\x894\x1f'
retired journey
